# 5章 顧客の退会を予測する１０本ノック

引き続き、スポーツジムの会員データを使って顧客の行動を分析していきます。  
３章では顧客の全体像を把握し、4章では数ヶ月利用している顧客の来月の利用回数の予測を行いました。   
ここでは、教師あり学習の分類を用いて、顧客の退会予測を取り扱います。

### ノック41：データを読み込んで利用データを整形しよう

In [3]:
import pandas as pd
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

In [4]:
# 当月(count_0)と1ヶ月前(count_1)の利用履歴を予測に使用
year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

C:\Users\81808\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### ノック42：退会前月の退会顧客データを作成しよう

In [5]:
# 例: 2018年8月に退会申請（exit_date)、9月(end_date)まで会員、10月は退会済
# 7月のデータから8月に退会申請する確率を後述のモデルで予測
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])

# 退会申請月（exit_date=end_date1ヶ月前の年月）を取得
from dateutil.relativedelta import relativedelta
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
print(len(uselog)) # 退会していない顧客も含んだ、１回以上顧客だった数
exit_uselog.head()

C:\Users\81808\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\81808\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\81808\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-ve

33851


C:\Users\81808\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# 退会申請した（exit_dateがある）顧客のみ抽出
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog)) # 退会者データ数
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


### ノック43：継続顧客のデータを作成しよう

In [7]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
# name列が欠損しているデータ（退会顧客）を除去
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog)) # 継続顧客データ総数（顧客の重複あり）

33851
27422


In [8]:
# 継続顧客データをアンダーサンプリング（顧客あたり１件のみランダム抽出）
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True) # データシャッフル
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id") # 最初に出現したIDのみ取得
print(len(conti_uselog)) # 解析に使う対照群
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201810,TS883038,6,6.0,XXXXX,C01,F,2017-06-01,NaN,CA1,...,オールタイム,10500.0,通常,5.416667,6.0,7.0,4.0,1.0,2019-04-30,22.0
1,201806,PL381578,8,7.0,XXXXX,C01,M,2016-09-01,NaN,CA1,...,オールタイム,10500.0,通常,5.166667,5.0,8.0,2.0,1.0,2019-04-30,31.0
2,201902,HI598974,6,7.0,XXX,C03,M,2015-05-01,NaN,CA1,...,ナイト,6000.0,通常,5.083333,5.0,7.0,3.0,1.0,2019-04-30,47.0
3,201901,PL459045,7,2.0,XXXX,C01,M,2016-05-01,NaN,CA1,...,オールタイム,10500.0,通常,5.000000,5.5,8.0,2.0,1.0,2019-04-30,35.0
4,201902,OA691219,8,10.0,XXXXX,C01,F,2018-12-06,NaN,CA3,...,オールタイム,10500.0,入会費無料,9.000000,9.0,10.0,8.0,1.0,2019-04-30,4.0


In [9]:
# 継続顧客データ（n=2842）と退会顧客データ（n=1104）を結合
predict_data = pd.concat([conti_uselog, exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


C:\Users\81808\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,calc_date,campaign_id,campaign_name,class,class_name,count_0,count_1,customer_id,end_date,exit_date,...,max,mean,median,membership_period,min,name,price,routine_flg,start_date,年月
0,2019-04-30,CA1,通常,C01,オールタイム,6,6.0,TS883038,NaN,NaN,...,7.0,5.416667,6.0,22.0,4.0,XXXXX,10500.0,1.0,2017-06-01,201810
1,2019-04-30,CA1,通常,C01,オールタイム,8,7.0,PL381578,NaN,NaN,...,8.0,5.166667,5.0,31.0,2.0,XXXXX,10500.0,1.0,2016-09-01,201806
2,2019-04-30,CA1,通常,C03,ナイト,6,7.0,HI598974,NaN,NaN,...,7.0,5.083333,5.0,47.0,3.0,XXX,6000.0,1.0,2015-05-01,201902
3,2019-04-30,CA1,通常,C01,オールタイム,7,2.0,PL459045,NaN,NaN,...,8.0,5.000000,5.5,35.0,2.0,XXXX,10500.0,1.0,2016-05-01,201901
4,2019-04-30,CA3,入会費無料,C01,オールタイム,8,10.0,OA691219,NaN,NaN,...,10.0,9.000000,9.0,4.0,8.0,XXXXX,10500.0,1.0,2018-12-06,201902


### ノック44：予測する月の在籍期間を作成しよう

In [ ]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

C:\Users\81808\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### ノック45：欠損値を除去しよう

In [9]:
predict_data.isna().sum() # 先月（count_1）の利用なしの顧客が246人

calc_date               0
campaign_id             0
campaign_name           0
class                   0
class_name              0
count_0                 0
count_1               276
customer_id             0
end_date             2842
exit_date            2842
gender                  0
is_deleted              0
max                     0
mean                    0
median                  0
membership_period       0
min                     0
name                    0
price                   0
routine_flg             0
start_date              0
年月                      0
period                  0
now_date                0
dtype: int64

In [10]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

calc_date               0
campaign_id             0
campaign_name           0
class                   0
class_name              0
count_0                 0
count_1                 0
customer_id             0
end_date             2618
exit_date            2618
gender                  0
is_deleted              0
max                     0
mean                    0
median                  0
membership_period       0
min                     0
name                    0
price                   0
routine_flg             0
start_date              0
年月                      0
period                  0
now_date                0
dtype: int64

### ノック46：文字列型の変数を処理できるように整形しよう

In [11]:
# is_deleted = 0: 継続、1: 退会（教師データ）
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
1,通常,オールタイム,F,8.0,1.0,18,0.0
2,通常,ナイト,F,6.0,1.0,24,0.0
3,通常,オールタイム,F,9.0,1.0,7,0.0
4,入会費半額,ナイト,M,6.0,1.0,2,0.0
5,通常,ナイト,M,8.0,1.0,20,0.0


In [12]:
# ダミー変数化
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
1,8.0,1.0,18,0.0,0,0,1,1,0,0,1,0
2,6.0,1.0,24,0.0,0,0,1,0,0,1,1,0
3,9.0,1.0,7,0.0,0,0,1,1,0,0,1,0
4,6.0,1.0,2,0.0,1,0,0,0,0,1,0,1
5,8.0,1.0,20,0.0,0,0,1,0,0,1,0,1


In [13]:
# 冗長な列を削除
del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]
predict_data.head()
predict_data.to_csv("kadai2.csv")

### ノック47：決定木を用いて退会予測モデルを作成してみよう

### ノック48：予測モデルの評価を行ない、モデルのチューニングをしてみよう

### ノック49：モデルに寄与している変数を確認しよう

### ノック50：顧客の退会を予測しよう